In [3]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['font.family'] = 'IPAPGothic' #全体のフォントを設定

%matplotlib inline

# 例)どんぶり専門店『丼兵衛』

## 評価値行列

 - $\boldsymbol {\chi} = \left\{ 1,\dots, 4 \right\}$
 - $\boldsymbol {y} = \left\{ 1,\dots,4 \right\}$
 - 3段階の採点法 : $$

In [11]:
R_arr = np.array([
    [1, 3, np.nan,  3],
    [np.nan, 1, 3, np.nan],
    [2, 1, 3, 1],
    [1, 3, 2, np.nan]
])

R_df = pd.DataFrame(
    R_arr, 
    columns=['親子丼', '牛丼', '海鮮丼', 'カツ丼'], 
    index=['山田', '田中', '佐藤', '鈴木']
)
R_df.head()

,親子丼,牛丼,海鮮丼,カツ丼
山田,1.0,3.0,NaN,3.0
田中,NaN,1.0,3.0,NaN
佐藤,2.0,1.0,3.0,1.0
鈴木,1.0,3.0,2.0,NaN


上記の評価値行列から、「**田中$x=2$**」の「**親子丼$y = 1$**」への推定評価値$\hat {r}_{2,1}$を求める

## 類似度の計算
ユーザー間の類似度をピアソン相関で計算する

In [25]:
Rho_df = R_df.T.corr()
Rho_df.head()

,山田,田中,佐藤,鈴木
山田,1.0,NaN,-1.0,1.0
田中,NaN,1.0,1.0,-1.0
佐藤,-1.0,1.0,1.0,-0.5
鈴木,1.0,-1.0,-0.5,1.0


## 嗜好の予測

### 全評価済みアイテム嬢の平均評価値を計算

In [18]:
mean_r_ser = R_df.mean(axis=1)
mean_r_ser.head()

山田    2.333333
田中    2.000000
佐藤    1.750000
鈴木    2.000000
dtype: float64

### 嗜好の予測値を計算

In [20]:
def predict(corr_arr, R_df):
    """
    ユーザのアイテムへの嗜好を計算する関数.
    
    Parameters:
    ---------------
     - corr_arr : ピアソン相関行列.
         shape = (n, )
     - R_df : 評価値行列.
         shape = (n, m)
    """
    pass

In [125]:
# 式(9.1)

a = 1
y = 0
x = [0,2,3]

r_x_ser = pd.Series([])
for i in x:
    # ユーザiが評価したアイテムの集合
    y_i = R_df.iloc[i][~R_df.iloc[i].isnull()].index
    # ユーザaが評価したアイテムの集合
    y_a = R_df.iloc[a][~R_df.iloc[a].isnull()].index
    y_a_x = np.intersect1d(y_a, y_i)
    r_x = np.sum(R_df.iloc[i][y_a_x]) / len(y_a_x)
    r_x_ser[R_df.index[i]] = r_x

pread_R = mean_r_ser[a] + np.nansum(Rho_df.iloc[a,x] * (R_df.iloc[x,y] - r_x_ser)) / np.nansum(np.abs(Rho_df.iloc[a,x]))
print(pread_R)

2.75


この値は、最大値3にかなり近く、「田中」は「親子丼」が好きであると予測される

In [126]:
# 式(9.2)
pread_R = mean_r_ser[a] + np.nansum(Rho_df.iloc[a,x] * (R_df.iloc[x,y] - mean_r_ser[x])) / np.nansum(np.abs(Rho_df.iloc[a,x]))
print(pread_R)

2.625
